In [6]:
%pip install torch torchvision

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset  
import os 
from PIL import Image   


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [7]:
class MaskDataset(Dataset):
    def __init__(self,root_dir,transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label,category in enumerate(["with_mask","without_mask"]):
            category_path = os.path.join(root_dir,category)
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path,img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image,label
    

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

dataset_path = r"E:\mL\jupyter-notebook\Face masked detection\dataset\data"
dataset = MaskDataset(root_dir = dataset_path,transform=transform)
dataloader = DataLoader(dataset,batch_size=32,shuffle=True)

In [8]:
print(f"Total images in dataset: {len(dataset)}")

image, label = dataset[0]  # First image and label
print(f"Image shape: {image.shape}, Label: {label}")


Total images in dataset: 7553
Image shape: torch.Size([3, 128, 128]), Label: 0


In [27]:
import torch.nn as nn 
import torch.nn.functional as F

class MaskCNN(nn.Module):
    def __init__(self):
        super(MaskCNN,self).__init__()

        self.conv1 = nn.Conv2d(in_channels =3,out_channels=32,kernel_size = 3,padding=1)
        self.pool = nn.MaxPool2d(kernel_size = 2,stride =2)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)

        self.fc1 = nn.Linear(64*32*32,128) 
        self.fc2 = nn.Linear(128,2)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x= self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x)
        return x      

In [10]:
model = MaskCNN()

In [11]:
import torch.optim as optim 

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [12]:
import torch

# Automatically detect GPU (agar available ho to)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [13]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss =0.0
    total  = 0
    correct = 0

    for images, labels in dataloader:
        images,labels = images.to(device),labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
        _, predicted = torch.max(outputs.data, 1)  
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}, Accuracy: {(100 * correct / total):.2f}%")

e:\mL\jupyter-notebook\venv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/5], Loss: 0.3313, Accuracy: 86.57%
Epoch [2/5], Loss: 0.1891, Accuracy: 92.52%
Epoch [3/5], Loss: 0.1151, Accuracy: 95.42%
Epoch [4/5], Loss: 0.0755, Accuracy: 97.30%
Epoch [5/5], Loss: 0.0364, Accuracy: 98.72%


In [29]:
# Save entire model
torch.save(model.state_dict(), "face_mask_model.pth")
print("✅ Model saved successfully!")


✅ Model saved successfully!
